In [1]:
import re
from typing import Dict

import torch

In [2]:
MODEL_CHECKPOINT_PATH = (
    "/home/pranav-pc/projects/OpenTransformer/multiformer/blm-1024/checkpoints/last.ckpt"
)
model_dict = torch.load(MODEL_CHECKPOINT_PATH)

In [3]:
model_dict.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'hparams_name', 'hyper_parameters'])

In [4]:
# print(model_dict['hyper_parameters'])

In [5]:
for key in model_dict["state_dict"]:
    print(key)

tok_embd.weight
layers.0.norms.w
layers.0.attention.wq.weight
layers.0.attention.wk.weight
layers.0.attention.wv.weight
layers.0.attention.wo.weight
layers.0.mlp.linear1.weight
layers.0.mlp.linear2.weight
layers.0.mlp.linear3.weight
layers.1.norms.w
layers.1.attention.wq.weight
layers.1.attention.wk.weight
layers.1.attention.wv.weight
layers.1.attention.wo.weight
layers.1.mlp.linear1.weight
layers.1.mlp.linear2.weight
layers.1.mlp.linear3.weight
layers.2.norms.w
layers.2.attention.wq.weight
layers.2.attention.wk.weight
layers.2.attention.wv.weight
layers.2.attention.wo.weight
layers.2.mlp.linear1.weight
layers.2.mlp.linear2.weight
layers.2.mlp.linear3.weight
layers.3.norms.w
layers.3.attention.wq.weight
layers.3.attention.wk.weight
layers.3.attention.wv.weight
layers.3.attention.wo.weight
layers.3.mlp.linear1.weight
layers.3.mlp.linear2.weight
layers.3.mlp.linear3.weight
norm.w
output.weight


In [13]:
_FROM_HF = {
    "model.embed_tokens.weight": "tok_embd.weight",
    "model.layers.{}.self_attn.q_proj.weight": "layers.{}.attention.wq.weight",
    "model.layers.{}.self_attn.k_proj.weight": "layers.{}.attention.wk.weight",
    "model.layers.{}.self_attn.v_proj.weight": "layers.{}.attention.wv.weight",
    "model.layers.{}.self_attn.o_proj.weight": "layers.{}.attention.wo.weight",
    "model.layers.{}.self_attn.rotary_emb.inv_freq": None,
    "model.layers.{}.mlp.gate_proj.weight": "layers.{}.mlp.linear1.weight",
    "model.layers.{}.mlp.up_proj.weight": "layers.{}.mlp.linear3.weight",
    "model.layers.{}.mlp.down_proj.weight": "layers.{}.mlp.linear2.weight",
    "model.layers.{}.input_layernorm.weight": "layers.{}.norms.w2",
    "model.layers.{}.post_attention_layernorm.weight": "layers.{}.norms.w",
    "model.norm.weight": "norm.w",
    "lm_head.weight": "output.weight",
}
{v: k for k, v in _FROM_HF.items()}

In [16]:
{v: k for k, v in _FROM_HF.items()}

{'tok_embd.weight': 'model.embed_tokens.weight',
 'layers.{}.attention.wq.weight': 'model.layers.{}.self_attn.q_proj.weight',
 'layers.{}.attention.wk.weight': 'model.layers.{}.self_attn.k_proj.weight',
 'layers.{}.attention.wv.weight': 'model.layers.{}.self_attn.v_proj.weight',
 'layers.{}.attention.wo.weight': 'model.layers.{}.self_attn.o_proj.weight',
 None: 'model.layers.{}.self_attn.rotary_emb.inv_freq',
 'layers.{}.mlp.linear1.weight': 'model.layers.{}.mlp.gate_proj.weight',
 'layers.{}.mlp.linear3.weight': 'model.layers.{}.mlp.up_proj.weight',
 'layers.{}.mlp.linear2.weight': 'model.layers.{}.mlp.down_proj.weight',
 'layers.{}.norms.w': 'model.layers.{}.post_attention_layernorm.weight',
 'norm.w': 'model.norm.weight',
 'output.weight': 'lm_head.weight'}

In [7]:
_FROM_META = {
    "tok_embeddings.weight": "tok_embd.weight",
    "norm.weight": "norm.w",
    "output.weight": "output.weight",
    "layers.{}.attention.wk.weight": "layers.{}.attention.wq.weight",
    "layers.{}.attention.wq.weight": "layers.{}.attention.wk.weight",
    "layers.{}.attention.wv.weight": "layers.{}.attention.wv.weight",
    "layers.{}.attention.wo.weight": "layers.{}.attention.wo.weight",
    "layers.{}.attention_norm.weight": "layers.{}.norms.w2",
    "layers.{}.ffn_norm.weight": "layers.{}.norms.w",
    "layers.{}.feed_forward.w1.weight": "layers.{}.mlp.linear1.weight",
    "layers.{}.feed_forward.w2.weight": "layers.{}.mlp.linear2.weight",
    "layers.{}.feed_forward.w3.weight": "layers.{}.mlp.linear3.weight",
}

In [8]:
def _get_mapped_key(key: str, mapping_dict: Dict[str, str]) -> str:
    try:
        if "layers" in key:
            # Replace layer number with "{}" to create key for lookup
            abstract_key = re.sub(r"(\.\d+)", ".{}", key)
            layer_num = re.search(r"\d+", key).group(0)

            new_key = mapping_dict[abstract_key]

            new_key = new_key.format(layer_num)
        else:
            new_key = mapping_dict[key]
    except KeyError as e:
        raise Exception(
            f'Error converting the state dict. Found unexpected key: "{key}". '
            "Please make sure you're loading a checkpoint with the right format. "
        ) from e

    return new_key

In [9]:
def blm_to_hf(
    state_dict: Dict[str, torch.Tensor],
    num_heads: int = 12,
    num_kv_heads: int = 12,
    dim: int = 768,
):
    """

    Args:
        state_dict (Dict[str, torch.Tensor]): State dict in blm's format.
        num_heads (int): Number of heads in the model.
        num_kv_heads (int): Number of heads in the key/value projection layers.
        dim (int): Dimension of the model.

    Returns:
        Dict[str, torch.Tensor]: State dict in Meta's format.
    """
    converted_state_dict = {}
    inverted_mapping_dict = {v: k for k, v in _FROM_HF.items()}
    head_dim = dim // num_heads

    def _permute(t, n_heads):
        return (
            t.view(n_heads, head_dim // 2, 2, dim)
            .transpose(1, 2)
            .reshape((head_dim * n_heads), dim)
        )

    for key, value in state_dict.items():
        new_key = _get_mapped_key(key, inverted_mapping_dict)
        if "q_proj" in key:
            value = _permute(value, num_heads)
        elif "k_proj" in key:
            value = _permute(value, num_kv_heads)
        converted_state_dict[new_key] = value

    return converted_state_dict

In [19]:
PATH = "/home/pranav-pc/projects/OpenTransformer/multiformer/blm-1024/checkpoints/blm2hf/consolidated.00.pth"
# torch.save(blm_to_hf(model_dict['state_dict']),PATH)

In [33]:
from transformers import AutoConfig, LlamaConfig, LlamaModel, LlamaTokenizerFast

LlamaConfig()

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 32000
}

In [10]:
def meta_to_blm(state_dict: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    """
    Args:
        state_dict (Dict[str, torch.Tensor]): State dict in Meta's format.

    Returns:
        Dict[str, torch.Tensor]: State dict in blm's format.
    """
    converted_state_dict = {}
    for key, value in state_dict.items():
        if key not in ["rope.freqs"]:  # Skip loading the position embeddings
            new_key = _get_mapped_key(key, _FROM_META)
            converted_state_dict[new_key] = value

    return converted_state_dict

In [11]:
# llama_model = torch.load('/home/pranav-pc/projects/OpenTransformer/checkpoints/llama-2-7b/consolidated.00.pth')
# llama_model = meta_to_blm(llama_model)

In [12]:
# PATH = "/home/pranav-pc/projects/OpenTransformer/multiformer/blm-1024/checkpoints/llama/blm-llama-7b.pth"

# llama_model['pytorch-lightning_version'] = '2.3.0.dev20240318'
# llama_model['hparams_name'] = 'kwargs'
# from src.models.blm.config import ModelArgs
# llama_model['hyper_parameters'] = {'args': ModelArgs(vocab_size=32000, embedding_dim=4096, max_seq_len=4096, embedding_dropout=0.0, rms_norm_eps=1e-05, rope_scaling=1.0, rope_theta=10000.0, attention_bias=False, attention_dropout=0.0, num_attention_heads=32, num_key_value_heads=32, use_cache=True, use_sliding_window=True, residual_dropout=0.1, mlp_hidden_size=11008, mlp_dropout=0.0, num_layers=32, device='cpu', padding_idx=2),
#  'is_causal': True,
#  'attn_mask': None,
#  'lr': 0.0005,
#  'cosine_t_max': 1000}
# torch.save(llama_model, PATH)

In [13]:
def blm_to_meta(state_dict: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    """
    Args:
        state_dict (Dict[str, torch.Tensor]): State dict in blm's format.

    Returns:
        Dict[str, torch.Tensor]: State dict in Meta's format.
    """
    converted_state_dict = {}
    inverted_mapping_dict = {v: k for k, v in _FROM_META.items()}

    for key, value in state_dict.items():
        new_key = _get_mapped_key(key, inverted_mapping_dict)
        converted_state_dict[new_key] = value

    return converted_state_dict

In [14]:
# blm_to_meta(model_dict['state_dict'])

In [27]:
def hf_to_blm(
    state_dict: Dict[str, torch.Tensor],
    num_heads: int = 32,
    num_kv_heads: int = 32,
    dim: int = 4096,
    head_dim: int = None,
) -> Dict[str, torch.Tensor]:
    """
    Convert a state dict from HF's format to blm's format. State dicts
    from multiple checkpoint files should be consolidated into a single state dict
    before calling this function.

    Eg of HF-format state dict can be found in the ``meta-llama/Llama-2-7b-hf``
    repo in HF (https://huggingface.co/meta-llama/Llama-2-7b-hf).

    Args:
        state_dict (Dict[str, torch.Tensor]): State dict in Meta's format.
        num_heads (int): Number of heads in the model.
        num_kv_heads (int): Number of heads in the key/value projection layers.
        dim (int): Dimension of the model.
        head_dim (int): Dimension of the head. If not provided, it will be calculated
            as dim // num_heads.

    Returns:
        Dict[str, torch.Tensor]: State dict in blm's format.
    """
    converted_state_dict = {}
    if head_dim is None:
        head_dim = dim // num_heads

    def _permute(t, n_heads):
        return (
            t.view(n_heads, 2, head_dim // 2, dim)
            .transpose(1, 2)
            .reshape((head_dim * n_heads), dim)
        )

    for key, value in state_dict.items():
        if "rotary_emb.inv_freq" not in key:  # Skip loading the position embeddings
            new_key = _get_mapped_key(key, _FROM_HF)
            if "q_proj" in key:
                value = _permute(value, num_heads)
            elif "k_proj" in key:
                value = _permute(value, num_kv_heads)
            converted_state_dict[new_key] = value
    return converted_state_dict